#### 导入数据库并展示相关信息

In [2]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
dataset=Planetoid(root='data/Planetoid',name='Cora',transform=NormalizeFeatures())#transform预处理
print()
print(f'Dataset: {dataset}:')
print('================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of fetures: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data=dataset[0] #get the first graph object

print()
print(data)
print('=================')
#gather some statistics about the graph
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of trainning nodes: {data.train_mask.sum()}')
print(f'Training node label rate"{int(data.train_mask.sum())/data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Cora():
Number of graphs: 1
Number of fetures: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of trainning nodes: 140
Training node label rate"0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True


#### 可视化数据函数

In [4]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
def visualize(h,color):
    z=TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())
    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])
    plt.scatter(z[:,0],z[:,1],s=70,c=color,cmap="Set2")
    plt.show()